# 変更点

- 履歴商品とペア商品にそれぞれ重みづけ

In [83]:
FOLD = '_fold1'
PAIR_WEIGHT = 0.3

# Recommend Items Frequently Purchased Together
This notebook demonstrates how recommending items that are frequently purchased together is effective. The current best scoring public notebook [here][1] recommends to customers those customers' last purchases and scores public LB 0.020. In this notebook here, we will begin with that idea and add recommending items that are frequently purchased together with a customers' previous purchaes. This notebook improves the LB and scores LB 0.021. This notebook's strategy is as follows:
* recommend items previously purchased [idea here][1]
* recommend items that are bought together with previous purchases [idea here][2]
* recommend popular items [idea here][1]

[1]: https://www.kaggle.com/hengzheng/time-is-our-best-friend-v2
[2]: https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this

In [84]:
import pandas as pd

# Find Each Customer's Last Week of Purchases
Our final predictions will have the row order from of our dataframe. Each row of our dataframe will be a prediction. We will create the `predictionstring` later by `train.groupby('customer_id').article_id.sum()`. Since `article_id` is a string, when we groupby sum, it will concatenate all the customer predictions into a single string. It will also create the string in the order of the dataframe. So as we proceed in this notebook, we will order the dataframe how we want our predictions ordered.

In [85]:
train = pd.read_parquet(f'train{FOLD}.pqt')

tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)

Train shape: (5184732, 5)


# (1) Recommend Most Often Previously Purchased Items
Note that many operations in pd will shuffle the order of the dataframe rows. Therefore we need to sort afterward because we want the most often previously purchased items first. Because this will be the order of our predictons. Since we sort by `ct` and then `t_dat` will will recommend items that have been purchased more frequently first followed by items purchased more recently second.

In [86]:
tmp = train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
train = train.merge(tmp,on=['customer_id','article_id'],how='left')
train = train.sort_values(['ct','t_dat'],ascending=False)
train = train.drop_duplicates(['customer_id','article_id'])
train = train.sort_values(['ct','t_dat'],ascending=False)
train.head()

,t_dat,customer_id,article_id,max_dat,diff_dat,ct
1164094,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
80390,2018-10-04,4485518665254175540,557247001,2018-10-04,0,86
2171280,2020-03-06,-906958334866810496,852521001,2020-03-06,0,81
3496438,2020-07-06,3601599666106972342,685813001,2020-07-06,0,80
875159,2019-05-14,-4601407992705575197,695545001,2019-05-14,0,80


# (2) Recommend Items Purchased Together
In my notebook [here][1], we compute a dictionary of items frequently purchased together. We will load and use that dictionary below. Note that we use the command `drop_duplicates` so that we don't recommend an item that the user has already bought and we have already recommended above. We will need to use Pandas for some commands because RAPIDS pd doesn't have two conveinent commands, (1) create new column from dictionary map of another column (2) groupby aggregate strings sum.

We concatenate these rows after the rows containing customers' previous purchases. Therefore we will recommend previous items first and then items purchased together second. Note the trick to convert a column of int32 into a prediction string (using groupby agg str sum) is from notebook [here][2]

[1]: https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this
[2]: https://www.kaggle.com/hiroshisakiyama/recommending-items-recently-bought

In [87]:
# USE PANDAS TO MAP COLUMN WITH DICTIONARY
import pandas as pd, numpy as np
# train = train.to_pandas()
pairs = np.load(f'../input/hmitempairs/pairs_cudf{FOLD}.npy',allow_pickle=True).item()
train['article_id2'] = train.article_id.map(pairs)

In [88]:
# RECOMMENDATION OF PAIRED ITEMS
train2 = train[['t_dat','customer_id','article_id2','ct']].copy()
train2 = train2.loc[train2.article_id2.notnull()]
train2 = train2.drop_duplicates(['customer_id','article_id2'])
train2 = train2.rename({'article_id2':'article_id'},axis=1)
train2['weight'] = train2['ct'] * PAIR_WEIGHT

In [89]:
# CONCATENATE PAIRED ITEM RECOMMENDATION AND SORT BY WEIGHT
train = train[['t_dat','customer_id','article_id','ct']]
train['weight'] = train['ct'].astype('float')
train = pd.concat([train,train2],axis=0,ignore_index=True)
train = train.sort_values(['weight', 't_dat'], ascending=False)[['customer_id', 'article_id']]
train.article_id = train.article_id.astype('int32')
train = train.drop_duplicates(['customer_id','article_id'])

In [90]:
# CONVERT RECOMMENDATIONS INTO SINGLE STRING
train.article_id = ' 0' + train.article_id.astype('str')
preds = pd.DataFrame( train.groupby('customer_id').article_id.sum().reset_index() )
preds.columns = ['customer_id','prediction']
preds.head()

,customer_id,prediction
0,-9223352921020755230,0673396002 0812167004 0706016001 0812167002
1,-9223343869995384291,0908292002 0910601003 0903926002 0865929007 0...
2,-9223321797620987725,0580600006 0610776035 0688018003 0610776002
3,-9223319430705797669,0470985003 0562245001 0646592007 0504155001 0...
4,-9223308614576639426,0750423005 0750423001


# (3) Recommend Most Popular Items
After recommending previous purchases and items purchased together we will then recommend the 12 most popular items. Therefore if our previous recommendations did not fill up a customer's 12 recommendations, then it will be filled by popular items.

In [91]:
train = pd.read_parquet(f'train{FOLD}.pqt')
# train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
# train['customer_id'] = train['customer_id'].str[-16:].str.hex_to_int().astype('int64')
# train['customer_id'] = train['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
# train['article_id'] = train.article_id.astype('int32')
# train.t_dat = pd.to_datetime(train.t_dat)
# train = train[['t_dat','customer_id','article_id']]

train.t_dat = pd.to_datetime(train.t_dat)
max_dat = train.t_dat.max()
train['diff_dat'] = (max_dat - train['t_dat']).dt.days
train = train.loc[train.diff_dat <= 6]
top12 = ' 0' + ' 0'.join(train.article_id.value_counts().index.astype('str')[:12])
print("Top 12 popular items:")
print( top12 )

Top 12 popular items:
 0909370001 0865799006 0918522001 0924243001 0448509014 0751471001 0809238001 0918292001 0762846027 0809238005 0673677002 0923758001


# Write Submission CSV
We will merge our predictions onto `sample_submission.csv` and submit to Kaggle.

In [92]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
sub = sub[['customer_id']]
sub['customer_id_2'] = sub['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
sub = sub.merge(preds.rename({'customer_id':'customer_id_2'},axis=1),\
    on='customer_id_2', how='left').fillna('')
del sub['customer_id_2']
sub.prediction = sub.prediction + top12
sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:131]
sub.to_csv(f'../submissions/002_submission{FOLD}_pair_weight_{PAIR_WEIGHT}.csv',index=False)
sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0909370001 0865799006 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0706016001 0909370001 0865799006 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0794321011 0909370001 0865799006 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001 0730683001 0372860001 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0927530004 0791587015 08...
